# A look at solution data and processed variables

Once you have run a simulation the first thing you want to do is have a look at the data. Most of the examples so far have made use of PyBaMM's handy QuickPlot function but there are other ways to access the data and this notebook will explore them. First off we will generate a standard SPMe model and use QuickPlot to view the default variables.

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np
import os
import matplotlib.pyplot as plt

os.chdir(pybamm.__path__[0] + "/..")

# load model
model = pybamm.lithium_ion.SPMe()

# set up and solve simulation
simulation = pybamm.Simulation(model)
dt = 90
t_eval = np.arange(0, 3600, dt)  # time in seconds
solution = simulation.solve(t_eval)

quick_plot = pybamm.QuickPlot(solution)
quick_plot.dynamic_plot();

Behind the scenes the QuickPlot classed has created some processed variables which can interpolate the model variables for our solution and has also stored the results for the solution steps

In [ ]:
solution.data.keys()

In [ ]:
solution.data["Negative particle surface concentration [mol.m-3]"].shape

In [ ]:
solution.t.shape

Notice that the dictionary keys are in the same order as the subplots in the QuickPlot figure. We can add new processed variables to the solution by simply using it like a dictionary. First let's find a few more variables to look at. As you will see there are quite a few:

In [ ]:
keys = list(model.variables.keys())
keys.sort()
print(keys)

If you want to find a particular variable you can search the variables dictionary

In [ ]:
model.variables.search("time")

We'll use the time in hours

In [ ]:
solution["Time [h]"]

This created a new processed variable and stored it on the solution object

In [ ]:
solution.data.keys()

We can see the data by simply accessing the entries attribute of the processed variable

In [ ]:
solution["Time [h]"].entries

We can also call the method with specified time(s) in SI units of seconds

In [ ]:
time_in_seconds = np.array([0, 600, 900, 1700, 3000])

In [ ]:
solution["Time [h]"](time_in_seconds)

If the variable has not already been processed it will be created behind the scenes

In [ ]:
var = "X-averaged negative electrode temperature [K]"
solution[var](time_in_seconds)

In this example the simulation was isothermal, so the temperature remains unchanged.

## Saving the solution

The solution can be saved in a number of ways:

In [ ]:
# to a pickle file (default)
solution.save_data(
    "outputs.pickle",
    ["Time [h]", "Current [A]", "Voltage [V]", "Electrolyte concentration [mol.m-3]"],
)
# to a matlab file
# need to give variable names without space
solution.save_data(
    "outputs.mat",
    ["Time [h]", "Current [A]", "Voltage [V]", "Electrolyte concentration [mol.m-3]"],
    to_format="matlab",
    short_names={
        "Time [h]": "t",
        "Current [A]": "I",
        "Voltage [V]": "V",
        "Electrolyte concentration [mol.m-3]": "c_e",
    },
)
# to a csv file (time-dependent outputs only, no spatial dependence allowed)
solution.save_data(
    "outputs.csv", ["Time [h]", "Current [A]", "Voltage [V]"], to_format="csv"
)

## Stepping the solver

The previous solution was created in one go with the solve method, but it is also possible to step the solution and look at the results as we go. In doing so, the results are automatically updated at each step.

In [ ]:
dt = 360
time = 0
end_time = solution["Time [s]"].entries[-1]
step_simulation = pybamm.Simulation(model)
while time < end_time:
    step_solution = step_simulation.step(dt)
    print("Time", time)
    print(step_solution["Voltage [V]"].entries)
    time += dt

We can plot the voltages and see that the solutions are the same

In [ ]:
voltage = solution["Voltage [V]"].entries
step_voltage = step_solution["Voltage [V]"].entries
plt.figure()
plt.plot(solution["Time [h]"].entries, voltage, "b-", label="SPMe (continuous solve)")
plt.plot(
    step_solution["Time [h]"].entries, step_voltage, "ro", label="SPMe (stepped solve)"
)
plt.legend()

As a final step, we will clean up the output files created by this notebook:

In [ ]:
os.remove("outputs.csv")
os.remove("outputs.mat")
os.remove("outputs.pickle")

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()